<a href="https://colab.research.google.com/github/hautran2705/ML/blob/main/ML_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !zip -r /content/result.zip /content/result

In [ ]:
#@title import module
!pip install nsepy 
from nsepy import get_history as gh
import datetime as dt
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
tickerName='AAPL'
typeTrain='day'
numberOfInput=1
sNumberOfInput='1'
lenFile=373
numberOftrain=300
#300 day, 2350 hour
#price of change 373,2610


In [ ]:
#@title add data
df=pd.read_csv(tickerName+".csv")
print("Number of rows and columns:", df.shape)

In [ ]:
#@title getClose
df['Date'] = df.index
data = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close'])
data['Date'] = df['Date']
data['Open'] = df['Open']
data['High'] = df['High']
data['Low'] = df['Low']
data['Close'] = df['Close']

In [ ]:
#@title define train and test
train_set = data.iloc[:,1:2].values
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train_set)
X_train = []
y_train = []
for i in range(numberOfInput, numberOftrain):
    X_train.append(training_set_scaled[i-numberOfInput:i, 0])
    y_train.append(training_set_scaled[i, 0]) 
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
#@title create model
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 30, batch_size = 32)

In [ ]:
#@title read test data
testdataframe=pd.read_csv(tickerName+".csv")
testdataframe['Date'] = testdataframe.index
testdata = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close'])
testdata['Date'] = testdataframe['Date']
testdata['Open'] = testdataframe['Open']
testdata['High'] = testdataframe['High']
testdata['Low'] = testdataframe['Low']
testdata['Close'] = testdataframe['Close']
real_stock_price = testdata['Close']
dataset_total = pd.concat((data['Close'], testdata['Close']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(testdata) - numberOfInput:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(numberOfInput, lenFile+numberOfInput+1):
    X_test.append(inputs[i-numberOfInput:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
#@title use model
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
#@title visualize
plt.figure(figsize=(20,10))
plt.plot(real_stock_price, color = 'green', label = 'SBI Stock Price')
plt.plot(predicted_stock_price, color = 'red', label = 'Predicted SBI Stock Price')
plt.title('SBI Stock Price Prediction')
plt.xlabel('Trading Day')
plt.ylabel('SBI Stock Price')
plt.legend()
plt.show()

In [ ]:
a_file = open("/content/result/"+typeTrain+"/"+tickerName+"_"+sNumberOfInput+".txt", "w")
for row in predicted_stock_price:
    np.savetxt(a_file, row)
a_file.close()